In [1]:
#!connect signalr --kernel-name rover --hub-url https://pitop-diego.ngrok.io/kernelhub

In [1]:
#!rover
ResourceScanner.CaptureFromCamera(RoverBody.Camera);
ResourceScanner.UseUri(new Uri("https://lobe-diego.ngrok.io/predict/34bc1074-f2fd-48bd-b473-7d1c4b50cd34"));
//ResourceScanner.UseModel(new DirectoryInfo("/home/pi/dotnet_conf_resources/models/v5.3"));

In [1]:
#!rover
var scannedSectors = CameraSector
.CreateSectors(5,5, Angle.FromDegrees(-60),Angle.FromDegrees(60),Angle.FromDegrees(-15),Angle.FromDegrees(30))
.Distinct()
.ToArray();
CameraSector currentSector = null;

In [1]:
#!rover
var testSector = new CameraSector(Angle.FromDegrees(-10), Angle.FromDegrees(-50));
RoverBody.TiltController.GoToSector(testSector);
testSector.CapturedFrame = RoverBody.Camera.GetFrame().Focus();
testSector.ClassificationResults = ResourceScanner.AnalyseFrame(testSector.CapturedFrame);
testSector

In [1]:
#!rover
void ResetSectors(IEnumerable<CameraSector> sectors) {
    foreach (var sector in sectors){
        sector.Reset();
    }
}

void Resetstate() {
    ResetSectors(scannedSectors);
    currentSector = null;
    RoverBody.AllLightsOff();
    RoverBody.TiltController.Reset();
}

bool IsResource(ClassificationResults result) {
    return result.Prediction.Label.Contains("no energy") != true;
}

bool AllSectorScanned(IEnumerable<CameraSector> sectors) {
    return scannedSectors.All(v => v.Marked);
}

bool FoundResources(IEnumerable<CameraSector> sectors, int requiredCount){
    return sectors.Where(s => s.ClassificationResults!= null && IsResource(s.ClassificationResults))
    .Select(s => s.ClassificationResults.Prediction.Label)
    .Distinct()
    .Count() >= requiredCount;
}

bool AllSectorScannedOrResourceTarget(IEnumerable<CameraSector> sectors, int requiredCount) {
    return AllSectorScanned(sectors) || FoundResources(sectors, requiredCount);
}

bool AnalyseSectorForResource(CameraSector sector){
    if(sector?.CapturedFrame != null) {
        sector.ClassificationResults = ResourceScanner.AnalyseFrame(sector.CapturedFrame);
        if(sector.ClassificationResults!= null && IsResource(sector.ClassificationResults)) {
            return true;
        }
        else {
            return false;
        }       
    }
    
    return false;
}

In [1]:
#!rover
Microsoft.DotNet.Interactive.Formatting.Formatter.ListExpansionLimit = 25;
scannedSectors = CameraSector
.CreateSectors(5,5, Angle.FromDegrees(-60),Angle.FromDegrees(60),Angle.FromDegrees(-15),Angle.FromDegrees(30))
.Distinct()
.ToArray();

Reset the state, we track sectors being scanned and the resoruce found

In [1]:
#!rover
RoverBody.TiltController.Reset();

In [1]:
#!rover
Resetstate();

Let's capture the current sector frame

In [1]:
#!rover

RoverBrain.Perceive = (roverBody, now, token) => {
    if(currentSector != null && currentSector.CapturedFrame == null) {
        Task.Delay(500).Wait();
        currentSector.CapturedFrame = roverBody.Camera.GetFrame().Focus();
    }
};

Define the planning logic:
1. Terminate when all sector have been visited and image was aquired
2. Classify the current sector if needed
3. Blink lights if a classification has been aquired

In [1]:
#!rover

RoverBrain.Plan = (roverBody, now, token) => {

    if ( AllSectorScanned(scannedSectors)){
        roverBody.AllLightsOff();
        return PlanningResult.NoPlan;
    }  


    if (currentSector != null) {
       
        currentSector.Marked = true;
    }

    if (AnalyseSectorForResource(currentSector))
    {
        roverBody.BlinkAllLights();
    }else {
        roverBody.AllLightsOff();
    }
    
    currentSector = scannedSectors.FirstOrDefault(s => s.Marked == false);

    if(currentSector != null)
    {
        return PlanningResult.NewPlan;
    }

    roverBody.AllLightsOff();
    return PlanningResult.NoPlan;
};

Let's move the camera to the sector we planned to scan

In [1]:
#!rover

RoverBrain.Act = (roverBody, now, token) => { 
    if(currentSector != null){
        roverBody.TiltController.GoToSector(currentSector);
    }   
};

Let's explore the sectors

In [1]:
#!rover
scannedSectors
    .Where(s => s.ClassificationResults != null )
    .GroupBy(s => s.ClassificationResults.Prediction.Label)
    .Select(g => new { Label = g.Key, Count = g.Count()})


In [1]:
#!rover
scannedSectors.Count(s => s.Marked == true)

In [1]:
#!rover
scannedSectors.Where(s => s.Marked == true)

In [1]:
#!rover
scannedSectors

In [1]:
#!rover
currentSector = scannedSectors.FirstOrDefault(s => s.Marked == false);


In [1]:
#!rover
currentSector